# Horoscope Generator

Train a machine learning model and generate horoscopes based on dataset from horoscope.com

I followed this guide: https://www.kaggle.com/code/shivamb/beginners-guide-to-text-generation-using-lstms

This generator notebook will create models for each of the zodiac signs. 

First import modules:

In [2]:
import tensorflow as tf

# keras module for building LSTM 
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import utils as ku
from keras import models as KM
from keras import backend as KBE

# set seeds for reproducability
from numpy.random import seed
tf.random.set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

tf.__version__


'2.10.0'

# Import the dataset:

Read all of the json files for each horoscope sign

In [ ]:

import pandas as pd
import re

signNames = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]

megaCorpus = {}

for i in range(1, 13): # exclude aries since I've already created and saved a model for aries
    df = pd.read_json('dataset2/'+ signNames[i-1] +'.json')
    corpus = list(df["text"])
    megaCorpus[signNames[i-1]] = corpus
    inp_sequences, total_words = get_sequence_of_tokens(corpus)
    print(total_words)

print(megaCorpus.keys())

# Define functions for sequences, creating the model and generating text

In [27]:
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words, tokenizer

def generate_padded_sequences(input_sequences, total_words):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+ output_word
    return seed_text.title()

# Train the Models

In [10]:
generated_horoscopes = {}

def train_models():
    signNames = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]
    
    for i in range(12, 13):
        KBE.clear_session()
        
        tokenizer = Tokenizer()
            
        curr_sign = signNames[i-1]
        print(curr_sign)
        
        df = pd.read_json('dataset2/'+ curr_sign +'.json')
        corpus = list(df["text"])

        # generate sequences
        inp_sequences, total_words, tokenizer = get_sequence_of_tokens(corpus)
        print(total_words)

        # obtain predictors and label
        predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences, total_words)

        # create models
        model = create_model(max_sequence_len, total_words)
        model.summary()

        print('--')

        # train models
        model.fit(predictors, label, epochs=100, verbose = 5)

        print('--')
        # evaluate model
        results = model.evaluate(x=predictors, y=label)
        print(results)

        print('--')

        model.save("model2-" + signNames[i-1] + ".h5")
        print("model saved")

        print('--')

        generated_text = generate_text("today", 100, model, max_sequence_len, tokenizer)
        generated_horoscopes[curr_sign] = generated_text
        print(generated_text)
        
    return


# Generating text with the saved models

In [13]:
def horoscopes_generator():
    signNames = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]

    generated_horoscopes = {}

    for i in range(1, 13):
        KBE.clear_session()
        
        
        curr_sign = signNames[i-1]
        
        print(curr_sign)
        
        df = pd.read_json('dataset2/'+ curr_sign +'.json')
        corpus = list(df["text"])

        # generate sequences
        inp_sequences, total_words, tokenizer = get_sequence_of_tokens(corpus)
        print(total_words)

        # obtain predictors and label
        predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences, total_words)

        print('--')

        # load models
        model = KM.load_model("Models/model2-" + curr_sign + ".h5")
        print("model loaded: " + "model2-" + curr_sign + ".h5")
        # model.summary()

        print('--')

        generated_text = generate_text("today", 100, model, max_sequence_len, tokenizer)
        generated_horoscopes[curr_sign] = generated_text
        
    for i in range(1, 13): 
        print("Generated horoscope for " + signNames[i-1] + ": ")
        print(generated_horoscopes[signNames[i-1]])
        print('--')

    return generated_horoscopes

In [16]:
def generate_horoscope(sign, starter_text, length):
    signNames = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]

    KBE.clear_session()

    curr_sign = sign

    print("Daily Horoscope for: " + curr_sign)

    df = pd.read_json('dataset2/'+ curr_sign +'.json')
    corpus = list(df["text"])

    # generate sequences
    inp_sequences, total_words, tokenizer = get_sequence_of_tokens(corpus)
    
    # obtain predictors and label
    predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences, total_words)
    
    # load models
    model = KM.load_model("Models/model2-" + curr_sign + ".h5")
    
    generated_text = generate_text(starter_text, length, model, max_sequence_len, tokenizer)
    
    print(generated_text)
    
    return generated_text

In [10]:
horoscopes_generator()

Aries
2714
--
model loaded: model2-Aries.h5
--


2022-10-03 18:53:01.574554: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 18:53:01.615106: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Taurus
2685
--
model loaded: model2-Taurus.h5
--


2022-10-03 18:53:04.922929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 18:53:04.956653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [28]:
leohoroscope = generate_horoscope("Pisces", "Today", 100)

Daily Horoscope for: Pisces


2022-10-03 19:04:58.300941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 19:04:58.333220: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Today You Might Be Hit With Some Exciting News Pisces This Could Involve New People Or Equipment Coming Onto The Scene Will Be A Pep Just To Your Own Interesting Step Should Be A Few More Less Than Play At This Can Do Up Your Willingness To Stand Up For A Crowd And Events If You Arent Already At A Little Bit Of Your Life Realize Out The Skies Is A Grain Of Love And Love Off More Than Your More Than All Your Most On Your Body Are You Might Be A Reassuring Caregiver For Someone Close Other Even Healing


In [12]:
train_models()

Pisces
2670
Metal device set to: Apple M1


2022-10-03 13:45:57.930405: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-03 13:45:57.931469: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 90, 10)            26700     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2670)              269670    
                                                                 
Total params: 340,770
Trainable params: 340,770
Non-trainable params: 0
_________________________________________________________________
--
Epoch 1/100


2022-10-03 13:45:58.433863: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-03 13:45:58.891550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 13:45:59.051788: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 13:45:59.950887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78/100
Epoch 7

2022-10-03 14:09:47.231629: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 14:09:47.289082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


569/569 [==============================] - 8s 14ms/step - loss: 0.9178
0.9177550673484802
--
model saved
--


2022-10-03 14:09:55.605078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 14:09:55.754816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Today You Might Be Hit With Some Exciting News Pisces This Could Involve New People Or Equipment Coming Onto The Scene Will Be A Pep Just To Your Own Interesting Step Should Be A Few More Less Than Play At This Can Do Up Your Willingness To Stand Up For A Crowd And Events If You Arent Already At A Little Bit Of Your Life Realize Out The Skies Is A Grain Of Love And Love Off More Than Your More Than All Your Most On Your Body Are You Might Be A Reassuring Caregiver For Someone Close Other Even Healing
